The apporach in this notebook is to ignore before/after satellite images and perform binary classification on images containing flooded and not flooded regions.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from fastai.metrics import accuracy, KappaScore
from fastai.vision import *

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('/kaggle/input/louisiana-flood-2016/train.csv')
valid_df = pd.read_csv('/kaggle/input/louisiana-flood-2016/test.csv')

In [ ]:
train_df.head(), valid_df.head()

In [ ]:
train_df['Image ID'] = train_df['Image ID'].apply(lambda x: os.path.join('train', x))
valid_df['Image ID'] = valid_df['Image ID'].apply(lambda x: os.path.join('test', x))

In [ ]:
train_df.head(), valid_df.head()

For getting a benchmark accuracy lets combine train and valid data.

In [ ]:
df = pd.concat([train_df, valid_df])
df.shape, df.head()

Remove the images taken before/after the flood.

In [ ]:
df = df[df['Normal']==0]
df.shape, df.head()

In [ ]:
data_dir = Path('../input/louisiana-flood-2016/')

In [ ]:
data = ImageDataBunch.from_df(data_dir, df, valid_pct=0.2, 
                              label_col='Flooded', ds_tfms=None,
                              size=224).normalize(imagenet_stats)

In [ ]:
data.classes

In [ ]:
kappa = KappaScore()
kappa.weights = "quadratic"

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=[accuracy, kappa], model_dir=Path('/kaggle/working/'))

In [ ]:
learn.fit_one_cycle(8)
learn.recorder.plot_losses()

In [ ]:
learn.show_results()

We dont have many data for performing deeplearning. Even though the model was able to give high accuracy, there is a chance that it is overfitting.
May be we can use augmentation for generating more samples.